<a href="https://colab.research.google.com/github/RadhavaramHarika/Harika_INFO5731_Spring2020/blob/master/Harika_INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

In [237]:
import nltk
import csv,os,math,unicodedata,re
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter 
from textblob import TextBlob
import pandas as panda
from sklearn.feature_extraction.text import TfidfVectorizer
import string,io
from google.colab import drive,files

def reading_csv():
  drive.mount('/content/gdrive')

  os.chdir('/content/gdrive/My Drive/Colab Notebooks')

  with open('UserReviews.csv', 'r') as csvFile:
    reader = csv.reader(csvFile,delimiter = ',')
    for i in range(3):
      next(reader)
    dat_dict = {'Review Title':[],
                'Total Reviews':[],
                'Preprocessed Reviews':[]}
    for row in reader:
      dat_dict['Review Title'].append(row[0])
      dat_dict['Total Reviews'].append(row[1])
      dat_dict['Preprocessed Reviews'].append(re.sub(r'\W+',' ',row[2]).strip())
    csvFile.close()

    datafr = panda.DataFrame(dat_dict)
    datafr.to_csv('UserReview.csv')

  return datafr
  
reading_csv()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Review Title,Total Reviews,Preprocessed Reviews
0,"As a viewer that actually went to TIFF and witnessed this film and didn't want to believe the hype, it is an absolute MASTERPIECE and Phoenix is a certified legend.\n",I was a person that saw all the hype and claims of masterpiece as overreacting and overblown excitement for another Joker based film. I thought this looked solid at best and even a bit too pretent...,i person saw hype claim masterpiec overreact overblown excit anoth joker base film i thought look solid best even bit pretenti trailer say i incred wrong thi massiv achiev cinema that extrem rare ...
1,Outstanding movie with a haunting performance and best character development ever seen\n,"Every once in a while a movie comes, that truly makes an impact. Joaquin's performance and scenography in all it's brilliance. Grotesque, haunting and cringy. Hard to watch at times,... but so mes...",everi movi come truli make impact joaquin perform scenographi brillianc grotesqu haunt cringi hard watch time mesmer wont blink eye watch tragic serious funni moment emot rollercoast sometim multi...
2,Only certain people can relate\n,This is a movie that only those who have felt alone and isolated can truly relate to it. You understand the motive and you feel sorry for the character. A lot of people will see this movie and thi...,thi movi felt alon isol truli relat you understand motiv feel sorri charact a lot peopl see movi think encourag violenc but truli movi encourag everi one u becom better person treat everyon respec...
3,Perfect in every aspect.\n,"Truly a masterpiece, The Best Hollywood film of 2019, one of the Best films of the decade... And truly the Best film to bring a comic book so chillingly and realistically to real ife.\nRemarkable ...",truli masterpiec the best hollywood film one best film decad and truli best film bring comic book chillingli realist real ife remark direct cinematographi music act some peopl surpris find disturb...
4,The Hype is real\n,"Most of the time movies are anticipated like this they end up falling short, way short. Joker is the first time I was more than happy with the hype. Please ignore the complaints of ""pernicious vio...",most time movi anticip like end fall short way short joker first time i happi hype plea ignor complaint pernici violenc embarrass say least we havent seen comic movi real if ever deserv better cla...
...,...,...,...
95,Astonishing Masterpiece\n,"What an incredible ride this was. I was almost motionless throughout, watching in awe the performance of a lifetime - the transition from troubled man to monster, the hounding score which directly...",what incred ride i almost motionless throughout watch awe perform lifetim transit troubl man monster hound score directli drove goosebump cinematographi made feel like fli gotham look arthur maste...
96,Nonsense plot\n,Arthur Fleck lives with his invalid mother. He is suffering from mental illness possibly as a result of childhood abuse. Budget cutbacks put an end to his weekly counselling sessions and he goes o...,arthur fleck live invalid mother he suffer mental ill possibl result childhood abus budget cutback put end weekli counsel session goe med in real life anyon el would quickli becom non function see...
97,I feel like everyone is brain washed\n,I thought this movie was complete and utter garbage. I don't understand why people liked this movie so much. The character doesn't even stay true to the comics and the director flat out said it di...,i thought movi complet utter garbag i dont understand peopl like movi much the charact doesnt even stay true comic director flat said didnt made seem like part gotham fall littl boy laugh problem ...
98,I made an account just to rate my disappoint for this film.\n,"I was expecting a masterpiece and oscar worthy film from all the hype.\nI actually didn't enjoy watching this movie.\nthere was no message, it was slow and the joker turned out just to be the son ...",i expect masterpiec os

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [238]:
# Write your code here
from nltk import ngrams,FreqDist,bigrams
import pandas as panda
import string,io,os
from google.colab import drive,files
import spacy 
import numpy as nump 
import math 
from tqdm import tqdm 
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
!pip install -U textblob
!python -m textblob.download_corpora
from textblob import TextBlob

panda.set_option('display.max_colwidth', 200)
nlp = spacy.load("en_core_web_sm")  
def calculate_n_grams(data,n):
  n_grams = []
  for each in data['Preprocessed Reviews'][:10]:
    n_grams.append(ngrams(each.split(), n))
  return n_grams

def get_reviewData():
  data = panda.read_csv('UserReview.csv')
  return data

def freq_cal(n_grams):
  freq_ngram = []
  for each in n_grams:
    fdist = nltk.FreqDist(each)
    freq_ngram.append(list(fdist.most_common()))
  return freq_ngram

def count_prob(dataframe):
  data = dataframe['Preprocessed Reviews'][:10]
  uni_grams = [word_tokenize(each) for each in data]
  bi_grams = [list(ngrams(each,2)) for each in uni_grams]
  uni_frq = freq_cal(uni_grams)
  bi_freq = freq_cal(bi_grams)
 
  count_prob_bi = []

  for i in range(len(bi_freq)):
    prob_each = []
    for each in bi_freq[i]:
      c_b = int(each[1])
      w2 = each[0][0]
      for e in uni_frq[i]:
        if e[0]==w2:
          c_w2 = int(e[1])
      prob_each.append(c_b/c_w2)
    count_prob_bi.append(prob_each)
  return count_prob_bi

def getNounPhrases(dataframe):
  preproc_data = dataframe['Preprocessed Reviews']
  np_count = []
  for each_revw in preproc_data:
    blob = TextBlob(each_revw)
    np_words = blob.noun_phrases
    fdist = nltk.FreqDist(np_words)
    np_count.append(fdist.most_common(1))
  return np_count

def rel_NP_Freq(no_count_list,max_NP):
  all_rel_freq = []
  for i in range(len(no_count_list)):
    for each in no_count_list[i]:
      c_np_revw = int(each[1])
      for np in max_NP:
        if np[0] == each[0]:
          c_np_t = int(np[1])
      all_rel_freq.append(c_np_revw/c_np_t)
  return all_rel_freq

def getMaxNPFreq(dataframe):
  text = ' '.join([each for each in dataframe['Preprocessed Reviews']])
  blob = TextBlob(text)
  total_np = blob.noun_phrases
  fdist = nltk.FreqDist(total_np)
  return fdist.most_common()

datafr = get_reviewData()
trigrams = calculate_n_grams(datafr,3)
print("\nQuestion 1: Count all Tri-grams:\n")
print("Tri-gram\t\tCount")
for each in freq_cal(trigrams):
  for gram in each:
    print(str(gram[0])+"\t"+str(gram[1]))

list_prob = count_prob(datafr)
print("\nQuestion 2: Probabilities of all bi-grams:\n")
for each in list_prob:
  print(each)

print("\nQuestion 3: Relative Probabilities of each review Noun phrases to whole dataset:\n")
allNPDatast = getMaxNPFreq(datafr)
datafr['Frequency of total Noun Phrases'] = rel_NP_Freq(getNounPhrases(datafr),allNPDatast)
datafr.to_csv("UserReview.csv")
datafr

Requirement already up-to-date: textblob in /usr/local/lib/python3.6/dist-packages (0.15.3)
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.

Question 1: Count all Tri-grams:

Tri-gram		Count
('i', 'person', 'saw')	1
('person', 'saw', 'hype')	1
('saw', 'hype', 'claim'

,Unnamed: 0,Review Title,Total Reviews,Preprocessed Reviews,Frequency of total Noun Phrases
0,0,"As a viewer that actually went to TIFF and witnessed this film and didn't want to believe the hype, it is an absolute MASTERPIECE and Phoenix is a certified legend.\n",I was a person that saw all the hype and claims of masterpiece as overreacting and overblown excitement for another Joker based film. I thought this looked solid at best and even a bit too pretent...,i person saw hype claim masterpiec overreact overblown excit anoth joker base film i thought look solid best even bit pretenti trailer say i incred wrong thi massiv achiev cinema that extrem rare ...,1.000000
1,1,Outstanding movie with a haunting performance and best character development ever seen\n,"Every once in a while a movie comes, that truly makes an impact. Joaquin's performance and scenography in all it's brilliance. Grotesque, haunting and cringy. Hard to watch at times,... but so mes...",everi movi come truli make impact joaquin perform scenographi brillianc grotesqu haunt cringi hard watch time mesmer wont blink eye watch tragic serious funni moment emot rollercoast sometim multi...,1.000000
2,2,Only certain people can relate\n,This is a movie that only those who have felt alone and isolated can truly relate to it. You understand the motive and you feel sorry for the character. A lot of people will see this movie and thi...,thi movi felt alon isol truli relat you understand motiv feel sorri charact a lot peopl see movi think encourag violenc but truli movi encourag everi one u becom better person treat everyon respec...,1.000000
3,3,Perfect in every aspect.\n,"Truly a masterpiece, The Best Hollywood film of 2019, one of the Best films of the decade... And truly the Best film to bring a comic book so chillingly and realistically to real ife.\nRemarkable ...",truli masterpiec the best hollywood film one best film decad and truli best film bring comic book chillingli realist real ife remark direct cinematographi music act some peopl surpris find disturb...,1.000000
4,4,The Hype is real\n,"Most of the time movies are anticipated like this they end up falling short, way short. Joker is the first time I was more than happy with the hype. Please ignore the complaints of ""pernicious vio...",most time movi anticip like end fall short way short joker first time i happi hype plea ignor complaint pernici violenc embarrass say least we havent seen comic movi real if ever deserv better cla...,1.000000
...,...,...,...,...,...
95,95,Astonishing Masterpiece\n,"What an incredible ride this was. I was almost motionless throughout, watching in awe the performance of a lifetime - the transition from troubled man to monster, the hounding score which directly...",what incred ride i almost motionless throughout watch awe perform lifetim transit troubl man monster hound score directli drove goosebump cinematographi made feel like fli gotham look arthur maste...,0.333333
96,96,Nonsense plot\n,Arthur Fleck lives with his invalid mother. He is suffering from mental illness possibly as a result of childhood abuse. Budget cutbacks put an end to his weekly counselling sessions and he goes o...,arthur fleck live invalid mother he suffer mental ill possibl result childhood abus budget cutback put end weekli counsel session goe med in real life anyon el would quickli becom non function see...,0.166667
97,97,I feel like everyone is brain washed\n,I thought this movie was complete and utter garbage. I don't understand why people liked this movie so much. The character doesn't even stay true to the comics and the director flat out said it di...,i thought movi complet utter garbag i dont understand peopl like movi much the charact doesnt even stay true comic director flat said didnt made seem like part gotham fall littl boy laugh problem ...,0.333333
98,98,I made an account just to rate my disappoint for this film.\n,"I was expecting a masterpiece and oscar worthy film from all the hype.\nI actu

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [242]:
# Write your code here
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  

def calc_tf_Idf(dataframe):
  for each in dataframe['Preprocessed Reviews']:
    data = panda.value_counts(each.split(" ")).rename_axis('words').reset_index(name = 'tf')
  for i,word in data['words'].items():
    data.loc[i, 'idf'] = nump.log(dataframe.shape[0]/(len(dataframe[dataframe['Preprocessed Reviews'].str.contains(word)])))
   
  data['documents-terms weights'] = data['tf']*data['idf']
  mod_data = data.head().style.set_properties(subset=['documents-terms weights'],**{'font-weight':'bold','border-color':'olive'})
  return mod_data


print("Question 1: Document-terms weights ")
calc_tf_Idf(datafr)



Question 1: Document-terms weights 


,words,tf,idf,documents-terms weights
0,joker,2,0.415515,0.831031
1,endgam,2,3.50656,7.01312
2,i,1,0,0
3,smile,1,2.81341,2.81341
4,histori,1,3.50656,3.50656


In [241]:
def rankingCosine(dataframe):

  train_set = dataframe['Preprocessed Reviews']
  test_set = ["An Outstanding movie with a haunting performance and best charater development"]
  tfidf_vectorizer = TfidfVectorizer()
  
  tfidf_matrix = tfidf_vectorizer.fit_transform(train_set)
  newtfidf = tfidf_vectorizer.transform(test_set)
  cosines = cosine_similarity(newtfidf, tfidf_matrix)
  documents = [i for i in range(100)]
  ranking = [i for _,i in sorted(zip(cosines[0],documents))]
  return (cosines[0],ranking)

similarities = rankingCosine(datafr)
print("Question : 2\n")
print("Cosine Similarities:\n")
print(similarities[0])

print("\nRanking of the documents/reviews with cosine similarity\n")
ranking_doc = ["doc_"+str(i) for i in similarities[1]]
print(ranking_doc)

Question : 2

Cosine Similarities:

[0.05368092 0.04096223 0.         0.15337562 0.         0.
 0.19773887 0.         0.07000601 0.03616925 0.         0.03166815
 0.         0.         0.0690286  0.         0.04674544 0.
 0.         0.         0.         0.01765318 0.         0.
 0.03845179 0.04055542 0.         0.         0.         0.
 0.         0.         0.1468701  0.         0.02548643 0.02772897
 0.08000861 0.         0.12856114 0.         0.206941   0.09317536
 0.01632504 0.06082853 0.         0.         0.         0.
 0.         0.05423235 0.04055542 0.         0.         0.
 0.         0.         0.         0.1468701  0.         0.02548643
 0.02772897 0.08000861 0.         0.12856114 0.         0.206941
 0.09317536 0.01632504 0.06082853 0.         0.         0.
 0.         0.         0.05423235 0.04055542 0.         0.
 0.         0.         0.         0.         0.1468701  0.
 0.02548643 0.02772897 0.08000861 0.         0.12856114 0.
 0.206941   0.09317536 0.01632504 0.06082

# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: https://github.com/RadhavaramHarika/Harika_INFO5731_Spring2020/blob/master/UserReview.csv